In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
angle=0
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[1] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

In [3]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [4]:
# # making training data
# data = []
# for img in glob.glob("./images/fall/*.jpg"):
#     n= cv2.imread(img)
#     object_detector = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=15)
#     mask = object_detector.apply(n)
#     _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     try:
#         imgRGB = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
#         results = pose.process(imgRGB)
#     except:
#         pass    
#     try:
#         landmarks=results.pose_landmarks.landmark

# #pose estimation

#         shoulder= [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                     landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
    
#         hip = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
        
#         knee = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]

#         #angleRRR = calculate_angle(shoulder, hip, knee)
                

#         shoulderl = [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                     landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
        
#         hipl = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
#                landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
        
#         kneel = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]
        

#         #angleLLL = calculate_angle(shoulderl, hipl, kneel)
#         data.append([shoulder[0],shoulder[1],hip[0],hip[1],knee[0],knee[1],shoulderl[0],shoulderl[1],hipl[0],hipl[1],kneel[0],kneel[1],1])

#     except:
#         pass
# for img in glob.glob("./images/not-fall/*.jpg"):
#     n= cv2.imread(img)
#     object_detector = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=15)
#     mask = object_detector.apply(n)
#     _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     try:
#         imgRGB = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
#         results = pose.process(imgRGB)
#     except:
#         pass    
#     try:
#         landmarks=results.pose_landmarks.landmark

# #pose estimation

#         shoulder= [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                     landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
    
#         hip = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
        
#         knee = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]

#         #angleRRR = calculate_angle(shoulder, hip, knee)
                

#         shoulderl = [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                     landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
        
#         hipl = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
#                landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
        
#         kneel = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
#                 landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]
        

#         #angleLLL = calculate_angle(shoulderl, hipl, kneel)
        
#         data.append([shoulder[0],shoulder[1],hip[0],hip[1],knee[0],knee[1],shoulderl[0],shoulderl[1],hipl[0],hipl[1],kneel[0],kneel[1],0])
#         #data.append([angle1,angle2,0])

#     except:
#         pass
# df = pd.DataFrame(data,columns=['shoulderRx','shoulderRy','hipRx','hipRy','kneeRx','kneeRy','shoulderLx','shoulderLy','hipLx','hipLy','kneeLx','kneeLy','fall'])
# df = df.sample(frac = 1)
# df.to_csv('train_data.csv')
    

In [5]:
data_df = pd.read_csv('train_data.csv')
df = data_df[['shoulderRx', 'shoulderRy', 'hipRx', 'hipRy', 'kneeRx',
       'kneeRy', 'shoulderLx', 'shoulderLy', 'hipLx', 'hipLy', 'kneeLx',
       'kneeLy','fall']]

In [6]:
#training svm model
X = df.drop(['fall'],axis='columns')
y = df.fall
x_train, x_test, y_train, y_test= train_test_split(X, y, test_size= 0.2)  
modelKNN = KNeighborsClassifier(n_neighbors=5)
modelKNN.fit(x_train, y_train)
modelSVC = SVC()
modelSVC.fit(x_train,y_train)
modelRFC= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
modelRFC.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [ ]:
for file in os.listdir("DATASET"):
    path=os.path.join("DATASET", file)
# Create a VideoCapture object and read from input file
    cap= cv2.VideoCapture(path)
#to check if video opens
    if (cap.isOpened() == False):
        print("Error opening video  file")
#Converting input to grey
#Segmenting foreground
#Remove noise and shadow
#detecting objects
    object_detector = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=15)

#if video opens
    while (cap.isOpened()):

# Capture frame-by-frame
        ret, frame = cap.read()
        mask = object_detector.apply(frame)
        _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#converting to rgb for pose estimation
        try:
            imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)
        except:
            pass

        try:
            landmarks=results.pose_landmarks.landmark

#pose estimation

            shoulder= [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
                    landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]

            angle3 = calculate_angle(shoulder, hip, knee)

            shoulderl = [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hipl = [landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].x,
                   landmarks[mpPose.PoseLandmark.RIGHT_HIP.value].y]
            kneel = [landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value].y]

            angle4 = calculate_angle(shoulderl, hipl, kneel)

        except:
            pass

#pose connection
        if results.pose_landmarks:
        #mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
            for cnt in contours:
# Calculate area and remove small elements
                area = cv2.contourArea(cnt)

                if area > 600:
                    predictionResult1 = modelKNN.predict([[shoulder[0],shoulder[1],hip[0],hip[1],knee[0],knee[1],shoulderl[0],shoulderl[1],hipl[0],hipl[1],kneel[0],kneel[1]]])
                    predictionResult2 = modelSVC.predict([[shoulder[0],shoulder[1],hip[0],hip[1],knee[0],knee[1],shoulderl[0],shoulderl[1],hipl[0],hipl[1],kneel[0],kneel[1]]])
                    predictionResult3 = modelRFC.predict([[shoulder[0],shoulder[1],hip[0],hip[1],knee[0],knee[1],shoulderl[0],shoulderl[1],hipl[0],hipl[1],kneel[0],kneel[1]]])
                    x, y, w, h = cv2.boundingRect(cnt)

                    if predictionResult1 == 1:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                        cv2.putText(frame,'fall detected',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
                    else:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255,0), 2)
                    #if  0< calculate_angle(shoulderl, hipl, kneel) <45 or 0< calculate_angle(shoulder, hip, knee) <45 or 270< calculate_angle(shoulder, hip, knee) >45 or 270< calculate_angle(shoulderl, hipl, kneel) >45  or  calculate_angle(shoulderl, hipl, kneel) <90 or  calculate_angle(shoulder, hip, knee)<90 or  calculate_angle(shoulderl, hipl, kneel) <45  or calculate_angle(shoulder, hip, knee) <45:
                     #   cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                      #  cv2.putText(frame,'fall detected',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
                    #else :
                     #   cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255,0), 2)

        if ret == True:
#to display the video
            cv2.imshow("videos", frame)
        # Press Q on keyboard to  exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

# Break the loop
        else:
            break
            
cv2.destroyAllWindows()
            

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeigh

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier wa

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier wa

...

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was 

C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\nirbh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
